# Implementing Fine Tuning

In [23]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import models

In [4]:
# cpu or gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## prepare dataloader

In [5]:
# for data load
from utils.dataloader_image_classification import ImageTransform, make_datapath_list,  HymenopteraDataset

# アリとハチの画像へのファイルパスリストを作成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

# Datasetの作成
size = 224
mean = (0.485, 0.456, 0.406)  # 書き換えのないようにtupleで作成
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    train_list,  transform=ImageTransform(size, mean, std), phase='train')
val_dataset = HymenopteraDataset(
    val_list, transform = ImageTransform(size, mean,std), phase='val')

# Dataloaderの作成
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書オブジェクトにまとめる  なんのためだろう
dataloader_dict = {'train':train_dataloader, 'val':val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


## make model

In [13]:
# 学習済みvgg16をロード

# vggモデルのインスタンスを生成
use_pretrained = True  # flagはわかりやすいように
net = models.vgg16(pretrained=use_pretrained)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
# vgg16のclassifierの最後の出力層をアリとハチの２値分類になるように付け替える
net.classifier[6] = nn.Linear(in_features=4069, out_features=2)

# train mode
net.train()
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードへ設定しました。')

ネットワーク設定完了：学習済みの重みをロードし、訓練モードへ設定しました。


In [15]:
# 最終層がちゃんと変わっているか確認
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
# モード確認　train or eval
net.training

True

In [17]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [22]:
# fine tuningで学習させるパラメータをparams_to_updateの１−３に格納する（後で見返せるように）
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
# なぜ３つに分けるのか -> それぞれ異なる学習率を適応するから
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias']
update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print('params_to_update_1に格納：　', name)
    elif name in update_param_names_2:
        param.requires_garad = True
        params_to_update_2.append(param)
        print('params_to_update_2に格納：　', name)
    elif name in update_param_names_3:
        param.requires_garad = True
        params_to_update_3.append(param)
        print('params_to_update_3に格納：　', name)
    else:
        # 更新する予定のないものは勾配を持たないようにする
        param.requires_grad = False
        print('勾配計算なし。学習しない：', name)
      

params_to_update_1に格納：　 features.0.weight
params_to_update_1に格納：　 features.0.bias
params_to_update_1に格納：　 features.2.weight
params_to_update_1に格納：　 features.2.bias
params_to_update_1に格納：　 features.5.weight
params_to_update_1に格納：　 features.5.bias
params_to_update_1に格納：　 features.7.weight
params_to_update_1に格納：　 features.7.bias
params_to_update_1に格納：　 features.10.weight
params_to_update_1に格納：　 features.10.bias
params_to_update_1に格納：　 features.12.weight
params_to_update_1に格納：　 features.12.bias
params_to_update_1に格納：　 features.14.weight
params_to_update_1に格納：　 features.14.bias
params_to_update_1に格納：　 features.17.weight
params_to_update_1に格納：　 features.17.bias
params_to_update_1に格納：　 features.19.weight
params_to_update_1に格納：　 features.19.bias
params_to_update_1に格納：　 features.21.weight
params_to_update_1に格納：　 features.21.bias
params_to_update_1に格納：　 features.24.weight
params_to_update_1に格納：　 features.24.bias
params_to_update_1に格納：　 features.26.weight
params_to_update_1に格納：　 features.26.bias


In [24]:
# 最適化手法の設定
# 引数を辞書のリストにして渡すことで場合分けをしている
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [ ]:
# モデルを学習させる関数を作成

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=25):
    # 初期設定
    # gpu or cup
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：　', device)
    
    # ネットワークをgpuへ
    net.to(device)
    
    # ネットワークがある程度固定であれば高速化できる
    torch.backends.cudnn.benchmark = True
    
    # epochのグループ
    